In [22]:
def tofaktorDOE(k=2):
    # Lage designmatrise for et 2^k faktorielt forsøk med gjentak
    # Sjekk om input er gyldig for ISTx1001
    if k <= 1 or k > 5 :
        raise ValueError("Ugyldig antall faktorer k")

    n_forsok = (2**k)

    # Lage designmatrise
    design_matrix = np.zeros((n_forsok, k))
    for i in range(n_forsok):
        for j in range(k):
            design_matrix[i, j] = (i >> j) & 1
        
    design_matrix = np.where(design_matrix == 0, -1, design_matrix)

    return pd.DataFrame(design_matrix, columns = ["x" + str(i) for i in range(1, k+1)])

In [31]:
def hovedeffektsplott(data, k, observasjoner = True,
                     faktornavn = ["A", "B", "C", "D", "E"],
                     variabelnavn = ["x" + str(i) for i in range(1, 6)]):
    
    # Sjekk om input er gyldig for ISTx1001
    if k <= 1 or k > 5 :
        raise ValueError("Ugyldig antall faktorer k")
    
    farger = ['#377eb8', '#ff7f00', '#4daf4a','#f781bf', '#a65628']
    
    ymax = max(data['y'])+np.std(data['y'])/2
    ymin = min(data['y'])-np.std(data['y'])/2
    
    kolonnenavn = list(data.columns)
    
    fig, axes = plt.subplots(1, k, figsize=(8, 4))
    for i in range(len(axes)):
        sns.regplot(x=kolonnenavn[i], y="y", data=data, ci=None, scatter = observasjoner, ax=axes[i], color = farger[i])
        axes[i].set_ylim(ymin, ymax)
        axes[i].set_xticks([-1, 1])
        axes[i].set_xlabel(variabelnavn[i])
        axes[i].set_title(faktornavn[i])
    
    fig.suptitle("Hovedeffekter")
    plt.tight_layout()
    plt.show()

In [39]:
def samspillsplott(data, k, 
                  faktornavn = ["A", "B", "C", "D", "E"], 
                  variabelnavn = ["x" + str(i) for i in range(1, 6)]):
    
    # Sjekk om input er gyldig for ISTx1001
    if k <= 1 or k > 5 :
        raise ValueError("Ugyldig antall faktorer k")
    
    farger = ['black', '#e41a1c']
    
    ymax = max(data['y'])
    ymin = min(data['y'])
    
    kolonnenavn = list(data.columns)
    
    if(k == 2):
        fig, axes = plt.subplots(1, k, figsize=(8, 4))
        i = 0
        for j in range(k):
            if(i==j):
                axes[j].axis('off')
                axes[j].annotate(variabelnavn[i]+" = -1", (0.5, 0.5), color = farger[0])
                axes[j].annotate(variabelnavn[i]+" = +1", (0.5, 0.7), color = farger[1])
            else:
                sns.regplot(x=kolonnenavn[j], y="y", data=data.loc[data[kolonnenavn[i]] == -1], 
                        ci=None, scatter = False, ax=axes[j], color = farger[0])
                sns.regplot(x=kolonnenavn[j], y="y", data=data.loc[data[kolonnenavn[i]] == 1], 
                        ci=None, scatter = False, ax=axes[j], color = farger[1])
                axes[j].set_xticks([-1, 1])
                axes[i].set_xlabel(variabelnavn[i])
                axes[j].set_title(faktornavn[i] + "*" + faktornavn[j])
                
        fig.suptitle("Samspillseffekter")
        plt.tight_layout()
        plt.show()
    else:
        fig, axes = plt.subplots(k-1, k, figsize=(8, 8))
    
        for i in range(k-1):
            for j in range(k):
                if(i==j):
                    axes[i,j].axis('off')
                    axes[i,j].annotate(variabelnavn[i]+" = -1", (0.5, 0.5), color = farger[0])
                    axes[i,j].annotate(variabelnavn[i]+" = +1", (0.5, 0.7), color = farger[1])
                elif (i > j):
                    axes[i,j].axis('off')
                else:
                    sns.regplot(x=variabelnavn[j], y="y", data=data.loc[data[variabelnavn[i]] == -1], 
                        ci=None, scatter = False, ax=axes[i,j], color = farger[0])
                    sns.regplot(x=variabelnavn[j], y="y", data=data.loc[data[variabelnavn[i]] == 1], 
                        ci=None, scatter = False, ax=axes[i,j], color = farger[1])
                    #axes[i,j].set_ylim(ymin, ymax)
                    axes[i,j].set_xticks([-1, 1])
                    axes[i,j].set_title(faktornavn[i] + " vs " + faktornavn[j])
                
        fig.suptitle("Samspillseffekter")
        plt.tight_layout()
        plt.show()

In [2]:
def DOEparetoplott(regresjonsmodell):
    # Extract estimated effects and variable names
    effekt = 2*regresjonsmodell.params[1:]  # Exclude the intercept
    abseffekt = abs(effekt)
    effektnavn = effekt.index
    
    # Create a DataFrame to hold the data
    data = pd.DataFrame({'Faktor': effektnavn, 'Effekt': effekt, 'Abseffekt': abseffekt})
    
    # Sort the DataFrame by absolute value of effects
    data = data.reindex(data['Abseffekt'].sort_values(ascending=False).index)
    
    # Create a new column to indicate the sign of the effect
    data['Fortegn'] = np.where(data['Effekt'] >= 0, 'Positiv', 'Negativ')
    
    if(regresjonsmodell.df_resid > 0):
        tcritical = stats.t.ppf(1-0.05/2,regresjonsmodell.df_model)
        effect_critical = tcritical*regresjonsmodell.bse[0]*2
        
        # Create the barplot with color-coded bars
        plt.figure(figsize=(6, 4))
        
        ax = sns.barplot(x='Abseffekt', y='Faktor', data=data, hue='Fortegn', dodge=False, palette={'Positiv': 'black', 'Negativ': 'darkgrey'})
        ax.axvline(x=effect_critical, color='red', linestyle='--', zorder=0)
        plt.xlabel('Effektestimater 2-nivå faktorielt forsøk')
        plt.ylabel(' ')
        plt.tight_layout()
        plt.legend(title='Fortegn')
        plt.show()
    else:
        # Create the barplot with color-coded bars
        plt.figure(figsize=(6, 4))
        sns.barplot(x='Abseffekt', y='Faktor', data=data, hue='Fortegn', dodge=False, palette={'Positiv': 'black', 'Negativ': 'darkgrey'})
        plt.xlabel('Effektestimater 2-nivå faktorielt forsøk')
        plt.ylabel(' ')
        plt.tight_layout()
        plt.legend(title='Fortegn')
        plt.show()
    